# Load segmentation, localise and save as h5

Segment a stack of images and then manually label a couple, then check how well the model segmented them.

In [1]:
import napari
from cellpose import models
from octopuslite import utils, tile
import numpy as np

import sys
sys.path.append('macrohet/')
from notify import send_sms

def view(img):
    return napari.Viewer().add_image(img)

from tqdm.auto import tqdm

import btrack
import dask.array as da

import torch
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA RTX A6000
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [2]:
image_dir = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/'
metadata_fn = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Index.idx.xml'
metadata = utils.read_harmony_metadata(metadata_fn)

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [5]:
metadata_path = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml'
assay_layout = utils.read_harmony_metadata(metadata_path, assay_layout=True)
utils.read_harmony_metadata(metadata_path, assay_layout=True)

Reading metadata XML file...
Extracting metadata complete!
Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

# Set up segmentation params

In [4]:
model = models.Cellpose(
                        #gpu=True, 
                        model_type='cyto', 
                        net_avg=True, 
                        device=torch.device('cuda')
                        )

# Iterate over positions and segment first frame

In [23]:
mask_t0_dict = dict()
for position, assay in tqdm(assay_layout.iterrows(), total = len(assay_layout)):
    row, column = position[0], position[1]
    input_image = tile.compile_mosaic(image_dir, 
                                 metadata, 
                                 row, 
                                 column, 
                                 set_channel=1, 
                                 set_plane = 'sum_proj',
                                 set_time = 1
                                 ).compute().compute().astype(np.uint16)
    masks, flows, styles, diams = model.eval(input_image, 
                                         batch_size = 32, 
                                         channels = [0,0], 
                                         diameter = 325, 
#                                          resample = True, 
                                         min_size = 2500, 
                                         progress = True)
    mask_t0_dict[(row, column)] = masks

  0%|          | 0/24 [00:00<?, ?it/s]

In [55]:
view(masks)

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'img' at 0x7fe499d82a00>

In [26]:
t0_dict = dict()
for position, assay in tqdm(assay_layout.iterrows(), total = len(assay_layout)):
    row, column = position[0], position[1]
    input_image = tile.compile_mosaic(image_dir, 
                                 metadata, 
                                 row, 
                                 column, 
#                                  set_channel=1, 
                                 set_plane = 'sum_proj',
                                 set_time = 1
                                 ).compute().compute().astype(np.uint16)
    t0_dict[(row, column)] = input_image

  0%|          | 0/24 [00:00<?, ?it/s]

In [56]:
mask_stack = np.stack([mask_t0_dict[key] for key in mask_t0_dict.keys()], axis = 0)
t0_stack = np.stack([t0_dict[key][0] for key in t0_dict.keys()], axis = 0)

keys = [key for key in mask_t0_dict.keys()]
print(keys)

[(3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9)]


In [58]:
mask_stack.shape

(24, 6048, 6048)

In [50]:
t0_stack.shape

(24, 1, 2, 6048, 6048)

In [62]:
v = napari.Viewer()
v.add_image(t0_stack, channel_axis=1, 
            colormap=['green', 'magenta'], 
            contrast_limits=[[100,6000],[0,3000]])
v.add_labels(mask_stack)

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Labels layer 'mask_stack' at 0x7fe4aa601160>

In [63]:
from skimage.io import imsave

In [66]:
imsave('/mnt/DATA/macrohet/segmentation/ground_truth/r03c04f0*p*1-ch1sk1fk1fl1.tiff', mask_stack[0])

/tmp/ipykernel_23651/1706692872.py:1: UserWarning: /mnt/DATA/macrohet/segmentation/ground_truth/r03c04f0*p*1-ch1sk1fk1fl1.tiff is a low contrast image
  imsave('/mnt/DATA/macrohet/segmentation/ground_truth/r03c04f0*p*1-ch1sk1fk1fl1.tiff', mask_stack[0])


In [92]:
%%time 
masks, flows, styles, diams = model.eval(input_images, 
                                         batch_size = 32, 
                                         channels = [0,0], 
                                         diameter = 250, 
#                                          resample = True, 
                                         min_size = 2500, 
                                         progress = True)

CPU times: user 2min 17s, sys: 36.2 s, total: 2min 53s
Wall time: 1min 30s


### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [24]:
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, 
                             column, 
                             #set_channel=1, 
                             set_plane = 'sum_proj',
#                              set_time = 1
                         )#.astype(uint8)

In [25]:
images

,Array,Chunk
Bytes,40.88 GiB,62.02 MiB
Shape,"(75, 2, 6048, 6048)","(1, 2, 2016, 2016)"
Count,16200 Tasks,675 Chunks
Type,uint64,numpy.ndarray


In [90]:
input_images = images[:,0,...]#.compute().compute().astype(np.uint16)
input_images

dask.array<getitem, shape=(75, 6048, 6048), dtype=uint64, chunksize=(1, 2016, 2016), chunktype=numpy.ndarray>

### Temporary: Load masks

In [97]:
masks_dict = np.load('segmentation/modified_mask_dict.npy', allow_pickle=True).item()
print(masks_dict.keys())

dict_keys([(200, 0.0), (200, 0.6), (250, 0.0), (300, 0.0), (300, 0.6), (200, 0.8)])


In [99]:
masks = masks_dict[(300,0.0)]

In [100]:
masks.shape

(75, 6048, 6048)

# Check masks 

and perform a quick post-segmentation cleaning on them

In [98]:
v = napari.Viewer()
v.add_image(images, 
            channel_axis = 1, 
            colormap=['green', 'magenta'], 
            contrast_limits=[[100,6000],[0,3000]])
# for key in masks_dict.keys():
#     v.add_labels(masks_dict[key][0], name = key)
v.add_labels(masks, name = 'masks')

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


## Post-segmentation editing

In [106]:
from skimage.morphology import remove_small_objects, remove_small_holes, binary_erosion, square

In [108]:
###Q: do i need to remove small objects? don't think so
###Q: do i need to erode? no as am not binarising...
cleaned_masks = list()
for i, mask in tqdm(enumerate(masks), total = len(masks)):
    mask = binary_erosion(mask, square(5))
    mask = remove_small_holes(mask, area_threshold=50)
    cleaned_masks.append(mask)
cleaned_masks = np.stack(cleaned_masks, axis = 0)

  0%|          | 0/75 [00:00<?, ?it/s]

In [113]:
view(masks[37])

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'img' at 0x7f09b5f966a0>

In [ ]:
masks = masks.compute()
masks

In [118]:
v = napari.Viewer()
v.add_image(images, 
            channel_axis = 1, 
            colormap=['green', 'magenta'], 
            contrast_limits=[[100,6000],[0,3000]])
# for key in masks_dict.keys():
#     v.add_labels(masks_dict[key][0], name = key)
# v.add_labels(masks, name = 'masks')
v.add_labels(cleaned_masks[37], name = 'masks')

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Labels layer 'masks' at 0x7f09cd7a5310>

In [122]:
v = napari.Viewer()
v.add_image(images[37,...], 
            channel_axis = 0, 
            colormap=['green', 'magenta'], 
            contrast_limits=[[100,6000],[0,3000]])
# for key in masks_dict.keys():
#     v.add_labels(masks_dict[key][0], name = key)
# v.add_labels(masks, name = 'masks')
v.add_labels(cleaned_masks[37], name = 'masks')

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Labels layer 'masks' at 0x7f09f1fc69d0>

#### Reorder the channel axis for localisation

In [6]:
gfp = images[:,0,...]
rfp = images[:,1,...]
images = da.stack([gfp,rfp], axis = -1)
images

,Array,Chunk
Bytes,40.88 GiB,31.01 MiB
Shape,"(75, 6048, 6048, 2)","(1, 2016, 2016, 1)"
Count,18900 Tasks,1350 Chunks
Type,uint64,numpy.ndarray


In [8]:
### convert to dask array so that it is compatible with images for localisation
masks = da.from_array(masks)

In [9]:
masks

dask.array<array, shape=(75, 6048, 6048), dtype=uint16, chunksize=(75, 864, 864), chunktype=numpy.ndarray>

# Localise raw masks

Include size filter and measurements of fluorescence

In [11]:
feat = [
      "area",
      "major_axis_length",
      "minor_axis_length",
      "orientation",
      "mean_intensity",
        ]

In [25]:
objects = btrack.utils.segmentation_to_objects(
    masks, 
    images,
    properties = tuple(feat),
    use_weighted_centroid = False, 
)

[INFO][2023/01/19 04:56:11 PM] Localizing objects from segmentation...
[INFO][2023/01/19 04:56:11 PM] Found intensity_image data
[INFO][2023/01/19 05:12:21 PM] Objects are of type: <class 'dict'>
[INFO][2023/01/19 05:12:21 PM] ...Found 89235 objects in 75 frames.


In [26]:
objects = [o for o in objects if o.properties['area'] > 2500]

#### Find the efd?

In [ ]:
### finding the EFD
for obj in tqdm(objects):
    ### extract the intensity image (1ch only)
    glimpse = obj.properties['intensity_image'][...,0]
    ### pad the glimpse to ensure only one object is identifiable
    glimpse = np.pad(glimpse, pad_width = 1)
    ### find the contours (zero because only one object)
    contours = skimage.measure.find_contours(glimpse, fully_connected='high', level = 0.5)[0]
    ### get the efd
    efd = elliptic_fourier_descriptors(contours, order=100, normalize=True)
#     obj.properties = {'efd': efd}
    flatten_efd = efd.flatten()
    obj.properties = {'efd flat': flatten_efd}

In [27]:
with btrack.dataio.HDF5FileHandler(
     'objects.h5', 'w', obj_type='obj_type_1',
) as hdf:
    hdf.write_segmentation(masks)
    hdf.write_objects(objects)

[INFO][2023/01/19 05:12:22 PM] Opening HDF file: objects.h5...
[INFO][2023/01/19 05:13:04 PM] Writing objects/obj_type_1
[INFO][2023/01/19 05:13:04 PM] Writing labels/obj_type_1
[INFO][2023/01/19 05:13:04 PM] Loading objects/obj_type_1 (40381, 5) (40381 filtered: None)
[INFO][2023/01/19 05:13:09 PM] Writing properties/obj_type_1/area (40381,)
[INFO][2023/01/19 05:13:09 PM] Writing properties/obj_type_1/major_axis_length (40381,)
[INFO][2023/01/19 05:13:09 PM] Writing properties/obj_type_1/minor_axis_length (40381,)
[INFO][2023/01/19 05:13:09 PM] Writing properties/obj_type_1/orientation (40381,)
[INFO][2023/01/19 05:13:09 PM] Writing properties/obj_type_1/mean_intensity-0 (40381,)
[INFO][2023/01/19 05:13:09 PM] Writing properties/obj_type_1/mean_intensity-1 (40381,)
[INFO][2023/01/19 05:13:09 PM] Closing HDF file: objects.h5


In [29]:
objects[0]

,ID,x,y,z,t,dummy,states,label,area,major_axis_length,minor_axis_length,orientation,mean_intensity-0,mean_intensity-1
0,0,257.838564,104.539205,0.0,0,False,7,5,36194,244.63143,204.217784,-0.625041,1751.473891,343.588302
